In [1]:
from langchain_core.runnables import RunnablePassthrough,RunnableLambda, Runnable, RunnableParallel,RunnableConfig,RunnableGenerator
from langchain_core.messages import AIMessage
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain.prompts import ChatPromptTemplate,SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser,StrOutputParser
from operator import itemgetter
from langchain.embeddings import SentenceTransformerEmbeddings
import json
from langchain_community.vectorstores import FAISS,Chroma
from operator import itemgetter
import time
import grandalf
from typing import Iterator,List,AsyncIterator
from langchain_core.runnables import ConfigurableField
from langchain.runnables.hub import HubRunnable

C:\Users\MSI 1\.conda\envs\oaivenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(find_dotenv("D:\LLM Courses\Master Langchain Udemy\.env"))

True

In [3]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llmGPT=ChatOpenAI(model="gpt-3.5-turbo")

In [5]:
prompt1=ChatPromptTemplate.from_template(
    template="What is the city {person} is from?"
)

prompt2=ChatPromptTemplate.from_template(
    template="What Country is the city {city} in? Respond in {language}")

In [6]:
chain1=prompt1|llmGemini|StrOutputParser()

In [7]:
chain2={"city":chain1,"language":itemgetter("language")}|prompt2|llmGemini|StrOutputParser()

In [9]:
print(chain2.invoke(input={"person":"Osama Bin Laden","language":"Bengali"}))

ওসামা বিন লাদেনের জন্মস্থান রিয়াদ, **সৌদি আরব**। 



In [10]:
chain2.get_graph().print_ascii()

               +------------------------------+        
               | Parallel<city,language>Input |        
               +------------------------------+        
                     ***               ***             
                  ***                     **           
                **                          ***        
  +--------------------+                       **      
  | ChatPromptTemplate |                        *      
  +--------------------+                        *      
             *                                  *      
             *                                  *      
             *                                  *      
+------------------------+                      *      
| ChatGoogleGenerativeAI |                      *      
+------------------------+                      *      
             *                                  *      
             *                                  *      
             *                                  

<h3>A More Complex Multiple Chain</h3>

In [19]:
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough,RunnableLambda, Runnable, RunnableParallel

llmGemini=ChatGoogleGenerativeAI(model="gemini-1.5-flash")

prompt1=ChatPromptTemplate.from_template(
    template="generate a {attribute} color. Return the name of the color and nothing else:"
)

prompt2=ChatPromptTemplate.from_template(
    template="what is a fruit of color: {color}. Return the name of the fruit and nothing else:"
)

prompt3=ChatPromptTemplate.from_template(
    template="what is a country with a flag that has the color: {color}. Return the name of the country and nothing else:"
)

prompt4=ChatPromptTemplate.from_template(
    template="What is the color of {fruit} and the flag of {country}?"
)

In [20]:
colorChain={"attribute":RunnablePassthrough()} | prompt1 | llmGemini | StrOutputParser()

fruitChain={"color":colorChain}|prompt2|llmGemini|StrOutputParser()

countryChain={"color":colorChain}|prompt3|llmGemini|StrOutputParser()

fruitAndCountryChain=RunnableParallel({"fruit":fruitChain,"country":countryChain})|prompt4|llmGemini|StrOutputParser()

In [21]:
print(fruitAndCountryChain.invoke(input={"attribute":"Yellow"}))

* **Banana:** Yellow
* **Flag of Ukraine:** Blue and yellow 



In [22]:
fruitAndCountryChain.get_graph().print_ascii()

                  +------------------------------+                   
                  | Parallel<fruit,country>Input |                   
                  +------------------------------+                   
                      ***                  ***                       
                  ****                        ****                   
                **                                **                 
      +-------------+                         +-------------+        
      | Passthrough |                         | Passthrough |        
      +-------------+                         +-------------+        
             *                                        *              
             *                                        *              
             *                                        *              
  +--------------------+                   +--------------------+    
  | ChatPromptTemplate |                   | ChatPromptTemplate |    
  +-----------------